# Preparación de la base de datos

Importar modulos

In [1]:
!pwd

/workspace/refactorizacion


In [2]:
from scripts.data import DinamicaMolecular
from scripts.data import Frame
from scripts.data import Posicion
from scripts.data import Fuerza
from scripts.data import ElementoQuimico
import scripts.data
import scripts.utils
import numpy as np

Factores de conversión

In [3]:
HARTREE_TO_EV=27.211386245981 #Original: 27.21138602
HA_BO_TO_EV_ANGS=51.4220675112 #Original: 51.422067137
BO_TO_ANGS=0.529177210544

Leer de archivo "input/input.data" y hacer las conversiones de unidades necesarias

In [4]:
# Abrir archivo
base_de_datos = DinamicaMolecular()
# Leer el archivo línea por línea
with open("input/input.data", "r") as archivo:

    for linea in archivo:
        # Convertir línea de texto a una lista Python
        linea = linea.split()

        # Detectar el inicio de un frame
        if linea[0] == "begin":
            frame_actual = Frame()  # Crear un nuevo frame vacío

        # Procesar información del átomo
        elif linea[0] == "atom":
            # Convertir valores de posición y fuerza a float
            x, y, z = map(float, linea[1:4])
            fx, fy, fz = map(float, linea[7:10])
            
            # Crear instancias de los elementos relacionados con el átomo
            elemento = ElementoQuimico(linea[4])  # Crear el objeto ElementoQuimico
            posicion = Posicion(x, y, z)* BO_TO_ANGS
            fuerza = Fuerza(fx, fy, fz) * HA_BO_TO_EV_ANGS

            # Agregar el átomo al frame actual
            frame_actual.agregar_atomo(elemento, posicion, fuerza)

        # Procesar información de la energía
        elif linea[0] == "energy":
            frame_actual.energia = float(linea[1]) * HARTREE_TO_EV

        # Finalizar el frame actual
        elif linea[0] == "end":
            # Agregar el frame a la base de datos
            base_de_datos.agregar_frame(frame_actual)

Hacer las modificaciones necesarias a la energía

In [5]:
for frame in base_de_datos.frames.values():
    # Calcular la suma de las energías atómicas de los elementos
    energia_total = sum(ElementoQuimico(e).energia_atomica() for e in frame.elementos)
    
    # Restar la energía atómica total de la energía actual del frame
    frame.energia -= energia_total  # Actualizar directamente el atributo

Guardar el dataset modificado en los formatos convenientes

In [6]:
# DinamicaMolecular.guardar_a_xyz(base_de_datos,"input/input.xyz")

In [7]:
#DinamicaMolecular.guardar_a_hdf5(base_de_datos,"input/input_LIACEI.h5")

In [8]:
DinamicaMolecular.guardar_a_TorchMDNet(base_de_datos,"input/input_torchmd-net.h5")

# Preparación del entrenamiento

Cargar la base de datos en caso de ser necesario

In [9]:
base_de_datos = DinamicaMolecular.cargar_desde_TorchMDNet("input/input_torchmd-net.h5")

Crear carpeta "output"

In [10]:
scripts.utils.crear_carpeta("output")

Carpeta creada o ya existente: output


Gestionar indices

In [11]:
numero_frames = base_de_datos.numero_frames
scripts.utils.procesar_y_guardar_csv_a_npz(numero_frames,train=0.81, test=0.1, val=0.09)
scripts.utils.cargar_o_generar_npz(numero_frames,train=0.81, test=0.1, val=0.09)

No se encontró ningún archivo .csv en la carpeta 'input'.
No se encontró un archivo .npz en la carpeta 'input'. Se generaron los ids de cada conjunto de datos y se guardaron en 'input/splits.npz'.


Obtener los indices de los conjuntos train, val y test de la base de datos

In [12]:
# Cargar los índices desde un archivo .npz
splits = np.load("input/splits.npz")
idx_train = np.sort(splits['idx_train'])
idx_val = np.sort(splits['idx_val'])
idx_test = np.sort(splits['idx_test'])

Separar los subconjuntos

In [13]:
# Obtener subconjuntos de la dinámica molecular
#subconjunto_train = DinamicaMolecular.subconjunto(base_de_datos, idx_train)
#subconjunto_val = DinamicaMolecular.subconjunto(base_de_datos, idx_val)
#subconjunto_test = DinamicaMolecular.subconjunto(base_de_datos, idx_test)

Guardar los subconjuntos en el formato adecuado

In [14]:
#DinamicaMolecular.guardar_a_xyz(subconjunto_train,"input/input_train.xyz")
#DinamicaMolecular.guardar_a_xyz(subconjunto_val,"input/input_val.xyz")
#DinamicaMolecular.guardar_a_xyz(subconjunto_test,"input/input_test.xyz")